In [ ]:
from Sapphire.Utilities.Building import nanoico #Let's build a small example nanocluster
nanoico.Nanoalloy.create_alloy_ico(['Au', 'Au', 'Au', 'Au', 'Au'], 6, 4.07, 'Au561.xyz')


Using the building tools in Sapphire, we are able to construct a small Au icosahedron of 561 atoms.
We may now proceed to perform some targetted alanysis on this object.

In [ ]:
from Sapphire.Post_Process import Adjacent,Kernels,DistFuncs #Some basic tools geometric analysis
from ase.io import read #Whilst this is a part of the core Sapphire setup, we can demonstrate its uitility more easily here

au = read('Au561.xyz')
pos = au.positions
dist = DistFuncs.Euc_Dist(pos) #Get a list of pairwise distances.
#Now why don't we visualise the pair distances as a distribution?
import matplotlib.pyplot as plt
a,b,c = plt.hist(dist, density=True, bins = 200)

This does not look particularly helpful, and so we shall make use of Sapphire's Kernel Density Estimators (KDEs) to get a smoother approximation on the pair distance distribution function.

We primarily advocate for the use of the Gaussian kernels, though others are supported.
A bandwidth of 0.05 should be adequate to acquire a good balance between smoothness and detail in the resultant distribution.

As a default, Sapphire only considers the first 6 Å of pair distances for the sake of speed when computing the full KDE. However, this is something which is easily varied, as explored in the PDDF tutorial.

In [ ]:
K = Kernels.Gauss(dist, 0.05)

In [ ]:
fig,ax = plt.subplots()
ax.plot(K.Space, K.Density, color = 'k', label = 'Gaussian KDE')
ax1 = ax.twinx()
a,b,c = ax1.hist(dist, density=True, bins = 200, color = 'r', label = 'Raw Histogram')
plt.xlim(0,6)
ax.set_xlabel('Pair distance (Å)')
ax.set_ylabel('Freq arb. un.')
fig.legend()

As we can see, there is a distinct advantage in using the KDE method to compute the pair distance distribution function over the raw histogram. Namely, we are able to extract information more easily, and being an analytic function, taking derivatives to find minima is simple!


Next, we compute the adjacency matrix which is stored as a sparse scipy matrix to save on memory overheads. This simply requires the atomic positions, the pair distances we computed earlier, and the first minimum of the pair distance distribution function.

In [ ]:
A = Adjacent.Adjacency_Matrix(Positions=pos, Distances=dist, R_Cut=K.R_Cut)
adj = A.ReturnAdj()

We shall introduce the CNA function by running through the main workflow explicitly before hiding much of the machinary behind the curtain of python.

In [ ]:
from Sapphire.CNA.FrameSignature import CNA
Sigs = {} #We shall not presupose the existence of any CNA signatures.
for i, atom in enumerate(au): #Iterate over all atoms in the cluster 
    cna = CNA(adj = adj)
    cna.NN(atom) #Aquire the nearest neighbours to the reference atom being considered
    for neigh in cna.neigh:
        sig = tuple((cna.R(atom,neigh), cna.S(), cna.T()))
        try:
            Sigs[sig]+=1
        except KeyError:
            print(sig)
            Sigs[sig] = 1

As we can see, there are 5 CNA signatures associated with the Ih structure which have been identified. This is indeed as it should be. 

555 - indicative of a 5-fold symmetry axis.
422 & 421 - Indicative of FCC environment
322 & 321 are generally representative of surfaces with (111) Miller indices. This shall be expanded on later in the tutorial.

With our signatures collected, we may now evaluate their distribution.

In [ ]:
import numpy as np
xaxis = np.arange(0,len(Sigs)) #Define the range for the x axis
yaxis = np.array([x for x in Sigs.values()]) #Set the y axis values from the dictionary
xlabels = [ str(x) for x in Sigs.keys() ] #Create a list of labels for the bar chart

plt.bar(xaxis,yaxis)

plt.xticks(xaxis,xlabels, rotation = 45)
plt.xlabel('CNA signature')
plt.ylabel('Counts')